#Linear regression from scratch:

1- Numerical Approach:
(Batch Gradient Descent, Stochatics Gradient Descent)

2- Analitycal Aproach


In [4]:
import numpy as np
import pandas as pd

In [5]:
#data generating

In [6]:
X_train=np.arange(0,20).reshape(-1,1)
X_test=np.arange(21,25).reshape(-1,1)
print(X_train.shape)
print(X_test.shape)

(20, 1)
(4, 1)


In [7]:
y_train=(2*X_train).reshape(-1,1)
true_y=2*X_test.reshape(-1,1)
print(y_train.shape)
print(true_y.shape)

(20, 1)
(4, 1)


In [107]:
class LinearRegression:
    def __init__(self,lr=0.01, max_iter=100,tolerence=10e-10,epsilon=10e-8):
        self.lr=lr
        self.max_iter=max_iter
        self.tolerence= tolerence
        self.epsilon=epsilon
        
    def loss(self,y,theta,x):
        '''
            This method compute the criteria (loss,cost) function of linear regression
        '''
        return (1/(2*len(x)))* np.sum((y - x@theta)**2)
    
    def get_gradient(self,y, theta, x):
        '''
            THis method compute the gradient of the loss function
        '''
        return (1/(len(x)))*np.sum((x@theta-y)*x, axis=0)
    
    #compute the Hessian 
    def hessian(self,X):
        return X.T@X + self.epsilon*np.eye(X.T.shape[0])
    
    #Batch gradient descent
    def fit_BGD(self,X,y):
        '''
            This method is the Batch gradient descent
        '''
        #make a copy of our data
        X_copy=X.copy()
        
        #add the intercept column
        intercept=np.ones((X_copy.shape[0],1))
        X_copy=np.concatenate((intercept, X_copy), axis=1)
        #print(X_copy.shape)
        #initialise the weight
        self.theta=np.zeros((X_copy.shape[1],1))
        #print(theta)
        #print('shape of theta',self.theta.shape)
        #print('shape of x',X_copy.shape)
        #print('shape of y',y.shape)
        diff=1
        current_iter=1
        while diff >= self.tolerence and current_iter<self.max_iter:
            grad = self.get_gradient(y, self.theta, X_copy)
            #print('gradient',grad.shape)
            prev_theta=self.theta.copy()
            self.theta = self.theta - self.lr*grad.reshape(-1,1)
            #print('shape of theta',self.theta.shape)
            cur_theta=self.theta.copy()
            diff=np.linalg.norm(prev_theta - cur_theta)
            current_iter+=1
            print('the loss is',self.loss(y,self.theta,X_copy))
    
    #stochastic gradient descent
    def fit_SGD(self,X,y):
        '''
            This method is the Stochastic gradient descent
        '''
        #make a coy of X
        X_copy=X.copy()
        #add the intercept
        
        intercept=np.ones((X_copy.shape[0],1))
        X_copy=np.concatenate((intercept,X_copy),axis=1)
        
        #initialize the weight
        self.theta=np.zeros((X_copy.shape[1],1))
        
        diff=1
        current_iter=1
        while diff >= self.tolerence and current_iter<self.max_iter:
            for i in range(self.max_iter):
                #shuffle the data
                shuffle_vector=np.random.permutation(X_copy.shape[0])
                #for one trainig set
                for j in shuffle_vector:
                    #compute the gradient
                    gradient= self.get_gradient(y[j],self.theta,X_copy[j])
                    prev_theta=self.theta.copy()
                    #update the weight
                    self.theta = self.theta - self.lr*gradient.reshape(-1,1)
                    cur_theta=self.theta.copy()
                    diff=np.linalg.norm(prev_theta - cur_theta)
                    current_iter+=1
                    print ('The loss is', self.loss(y[j],self.theta,X_copy[j]))

    #metod to solve analytically
    def fit_analytically(self,X,y):
        '''
            This method is the analytical approach
        '''
        X_copy=X.copy()
        intercept=np.ones((X_copy.shape[0],1))
        X_copy=np.concatenate((intercept,X_copy),axis=1)
        
        #self.theta = np.zeros((X_copy.shape[1],1))
        
        self.theta=np.linalg.solve(X_copy.T@X_copy,X_copy.T@y)
        
        print('The mimimum value of theta is \n',self.theta,'\nThe loss is ', self.loss(y, self.theta, X_copy))
    
    def newton_method(self,X,y):
        #make a coy of X
        X_copy=X.copy()
        #add the intercept
        
        intercept=np.ones((X_copy.shape[0],1))
        X_copy=np.concatenate((intercept,X_copy),axis=1)
        
        #initialize the weight
        self.theta=np.zeros((X_copy.shape[1],1))
        H=self.hessian(X_copy)
        inv_H=np.linalg.inv(H)
        diff=1
        current_iter=1
        while diff >= self.tolerence and current_iter<self.max_iter:
            grad=self.get_gradient(y, self.theta, X_copy)
            grad=grad.reshape(-1,1)
            prev_theta=self.theta.copy()
            self.theta = self.theta - inv_H@grad
            cur_theta=self.theta.copy()
            diff=np.linalg.norm(prev_theta - cur_theta)
            current_iter+=1
            print('the loss is',self.loss(y,self.theta,X_copy))
        #return 
        
     #this method return the prediction   
    def predict(self,X):
        '''
            This method takes a data and predict the target
        '''
        X_copy=X.copy()
        intercept=np.ones((X_copy.shape[0],1))
        print('shape of theta',self.theta.shape)
        #print('shape of intercept', intercept.shape)
        #print('shape of x copy', X_copy.shape)
        X_copy=np.concatenate((intercept,X_copy),axis=1)
        #print('shape of x_copy', X_copy.shape)
        return X_copy@self.theta
    
    #This method predict and  return the RMSE 
    def error_prediction(self,X,y):
        '''
            This method is doing the same things as predict but instead it returns the RMSE
            
            It was create just for the test puporse in the cross validation 
        '''
        
        X_copy=X.copy()
        intercept=np.ones((X_copy.shape[0],1))
        #print('shape of theta',self.theta.shape)
        #print('shape of intercept', intercept.shape)
        #print('shape of x copy', X_copy.shape)
        X_copy=np.concatenate((intercept,X_copy),axis=1)
        #print('shape of x_copy', X_copy.shape)
        predict=X_copy@self.theta
        return np.sqrt(np.mean(np.sum((predict - y)**2)))
        
            
        

In [108]:

linearReg=LinearRegression(lr=0.01,max_iter=100)

In [109]:
linearReg.newton_method(X_train,y_train)

the loss is 222.91750000094999
the loss is 201.18304375171476
the loss is 181.56769698669635
the loss is 163.8648465311918
the loss is 147.88802399503086
the loss is 133.46894165608416
the loss is 120.4557198451293
the loss is 108.71128716069248
the loss is 98.1119366629431
the loss is 88.54602283868348
the loss is 79.91278561225242
the loss is 72.12128901536516
the loss is 65.08946333664447
the loss is 58.74324066157195
the loss is 53.01577469729464
the loss is 47.84673666451232
the loss is 43.18167983990638
the loss is 38.97146605568158
the loss is 35.17174811540252
the loss is 31.74250267428604
the loss is 28.647608663665235
the loss is 25.854466819068058
the loss is 23.333656304308363
the loss is 21.058624814728045
the loss is 19.005408895373048
the loss is 17.152381528147274
the loss is 15.480024329218878
the loss is 13.970721957179576
the loss is 12.608576566408303
the loss is 11.379240351231998
the loss is 10.26976441703065
the loss is 9.268462386409654
the loss is 8.36478730377

----------------------------------training using analytically approach---------------------------

In [110]:
linearReg.fit_analytically(X_train,y_train)

The mimimum value of theta is 
 [[0.]
 [2.]] 
The loss is  0.0


----------------------------------training using Batch Gradient Descent---------------------------

In [111]:
linearReg.fit_BGD(X_train,y_train)

the loss is 14.506975000000006
the loss is 0.8548038012499992
the loss is 0.05312288510998401
the loss is 0.0060319596356099466
the loss is 0.003251177456302545
the loss is 0.003072401559057719
the loss is 0.0030465001025298363
the loss is 0.003029657884840527
the loss is 0.0030134295351188774
the loss is 0.002997318697365723
the loss is 0.0029812957897152634
the loss is 0.0029653586419381554
the loss is 0.002949506695751016
the loss is 0.002933739489828651
the loss is 0.0029180565708299155
the loss is 0.002902457488161144
the loss is 0.002886941793656533
the loss is 0.002871509041547091
the loss is 0.0028561587884468687
the loss is 0.002840890593340145
the loss is 0.0028257040175687347
the loss is 0.002810598624819336
the loss is 0.0027955739811111515
the loss is 0.0027806296547832738
the loss is 0.0027657652164822456
the loss is 0.002750980239149969
the loss is 0.0027362742980111382
the loss is 0.002721646970561236
the loss is 0.002707097836554288
the loss is 0.002692626477990942
the

----------------------------------training using Stochastic Gradient Descent---------------------------

In [112]:
linearReg.fit_SGD(X_train,y_train)

The loss is 3.062500000000006
The loss is 2.4641150625000083
The loss is 0.7403333316890622
The loss is 0.017768130198122196
The loss is 7.090920529319169e-05
The loss is 0.0004135866713285145
The loss is 0.03738585293975598
The loss is 0.040113949250310865
The loss is 0.01863034780099443
The loss is 0.6573198339981284
The loss is 0.0012721149839719872
The loss is 0.00015178205423946946
The loss is 0.031559501956435046
The loss is 0.0010581582042051355
The loss is 0.4213752174728712
The loss is 0.0011816922824713849
The loss is 0.6116693167130227
The loss is 0.0424507676714034
The loss is 0.3544498111156549
The loss is 0.016818290746982083
The loss is 0.006237839677006358
The loss is 0.8721798061170964
The loss is 0.16458557853658584
The loss is 0.019699240740737315
The loss is 0.12254212805094114
The loss is 0.34678915824966533
The loss is 0.20763428925703667
The loss is 0.6886281929757103
The loss is 0.010234723835838056
The loss is 0.1401729009933623
The loss is 0.011880552566814398

----------------------------------predict ---------------------------

In [113]:
predict=linearReg.predict(X_test)
print(linearReg.error_prediction(X_test,true_y))
print('\nOur prediction are\n',predict)
print('\nThe true value of y are\n',true_y)

shape of theta (2, 1)
6.338251239329626

Our prediction are
 [[39.16973201]
 [40.95017437]
 [42.73061674]
 [44.5110591 ]]

The true value of y are
 [[42]
 [44]
 [46]
 [48]]


----------------------------------We compute the root means square error---------------------------

In [114]:
np.sqrt(np.mean(np.sum((predict - true_y)**2)))

6.338251239329626

################################ K-fold cross validation ######################################################

In [115]:
class CrossValidation:
    def __init__(self,k=10, max_iteration=10):
        self.k=k
        self.max_iteration=max_iteration
    
    
    def split_train_test(self,data,step):
        
        '''
            This method split and prepare the trainning and the validation set.
            
            It takes the data that was shuffled and split using the k
        '''
        #print('data in split',data)
        #print('setp',step)
        data = data[:]
        
        data_test = data[step]
        X_test = data_test[0]
        y_test =data_test[1]
        
        X_train = []
        y_train =[]
        data_train=data.pop(step)
        
        for tup in data_train:
            X_train.append(tup[0])
            y_train.append(tup[1])
        return X_train, y_train, X_test, y_test
            
    def cross_validation_split(self,X,y):
        
        '''
            This method randomly split the data into k subset of size size_of_initial_data/k
        '''
        
        #size of each subset
        permut_index=np.random.permutation(len(X))
        X = X[permut_index]
        y = y[permut_index]
        size_subset=int(len(X)/self.k)
        split_data=[]
        
        for i in range(self.k):
            X_s = X[i*size_subset:(i+1)*size_subset]
            y_s = y[i*size_subset: (i+1)*size_subset]
            split_data.append((X_s,y_s))
        return split_data 
            
    
    def cross_validation(self, learning_rate, X, y):
        
        '''
            This method takes the list of model and the data, return the best model with his RMSE 
        '''
        
        #randomly split the data
        data_split = self.cross_validation_split(X,y)
        
        generalization_error=[]
        
        for alpha in learning_rate:
            errors=[]
            #get the model with learning rate alpha
            linearReg = LinearRegression(lr=alpha, max_iter=self.max_iteration)
            for i in range(self.k):
                #split the big splitting data into train and validation set
                #print('iteration',i)
                #print('my data is of lenght',len(data_split))
                X_train,y_train,X_test,y_test = self.split_train_test(data_split, i)
                #train the model
                linearReg.fit_SGD(np.array(X_train),np.array(y_train))
                er=linearReg.error_prediction(X_test,y_test)
                errors.append(er)
            mean_errors=np.mean(errors)
            generalization_error.append(mean_errors)
            model=learning_rate[np.argmin(generalization_error)]
        return {'Best_hyperparameter': model ,'RMSE': np.min(generalization_error) } 

In [116]:
cross= CrossValidation()

In [117]:
#d=cross.cross_validation_split(X_train,y_train)
cross.cross_validation([0.1,0.01,0.001],X_train,y_train)

The loss is 654.0806250000002
The loss is 11248988.987376563
The loss is 862274546.6453245
The loss is 14876628622213.414
The loss is 1140399515528596.2
The loss is 1.967511037326106e+19
The loss is 1.5082373718638676e+21
The loss is 2.602135166457948e+25
The loss is 1.9947219763332783e+27
The loss is 3.4414584194527205e+31
The loss is 2.638123041572859e+33
The loss is 4.551506856941665e+37
The loss is 2.0707091824423257e+41
The loss is 1.5873461017920188e+43
The loss is 4.614454801561945e+45
The loss is 7.961236962562096e+49
The loss is 6.10285527543772e+51
The loss is 1.052914788103156e+56
The loss is 4.790238460762279e+59
The loss is 3.6720590278045906e+61
The loss is 12488.0625
The loss is 7759.407656250003
The loss is 16314.154597265626
The loss is 3094821.253458291
The loss is 171770316.62006885
The loss is 104728407.70134397
The loss is 20050104285.84126
The loss is 12222740405.45031
The loss is 25698311702.459263
The loss is 4919315043941.572
The loss is 273034283226367.06
The 

{'Best_hyperparameter': 0.001, 'RMSE': 15.73424456091791}

Filter methode : Mutual Information, Kullblack leibler divergence

In [106]:
data=pd.read_csv("../Data/diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [40]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.002032,0.001594,0.001357,0.002219,0.000000,0.001368,0.001730,0.001959,0.003731
1,0.000339,0.000915,0.001244,0.001839,0.000000,0.001083,0.000969,0.001214,0.000000
2,0.002709,0.001971,0.001206,0.000000,0.000000,0.000948,0.001854,0.001253,0.003731
3,0.000339,0.000959,0.001244,0.001458,0.001534,0.001144,0.000461,0.000823,0.000000
4,0.000000,0.001476,0.000754,0.002219,0.002741,0.001754,0.006313,0.001293,0.003731


In [97]:
def KL(data,label,k):
    #normalize the data
    epsilon=10e-10
    data=data.copy()
    data_normalize=data/np.sum(data,axis=0)
    target=data_normalize[label].values
    data_feature=data_normalize.drop(label,axis=1)
    features=data_feature.columns
    print(features)
    kullblack=[]
    for feat in features:
        kl=np.sum(data_normalize[feat] * np.log((data_normalize[feat] + epsilon)/(target + epsilon)),axis=0)
        kullblack.append(kl)
    kullblack = np.array(kullblack)
    kullblack_index = np.argsort(kullblack)
    kullblack = kullblack[kullblack_index]
    features = features[kullblack_index]
    
    return pd.DataFrame(kullblack, index=features).iloc[:k,]
    #print(kullblack_index)
    #print(kullblack[kullblack_index])
        
    

In [98]:
KL(data,'Outcome', 2)

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')


,0
Pregnancies,7.801042
Glucose,7.946917
